## We will use xena's API to retrieve our data. 

In [1]:
import pandas as pd

In [4]:
import xenaPython as xena

In [5]:
help(xena)

Help on package xenaPython:

NAME
    xenaPython - Methods for querying data from UCSC Xena hubs

DESCRIPTION
    Data rows are associated with "sample" IDs.
    Sample IDs are unique within a "cohort".
    A "dataset" is a particular assay of a cohort, e.g. gene expression.
    Datasets have associated metadata, specifying their data type and cohort.
    
    There are three primary data types: dense matrix (samples by probes),
    sparse (sample, position, variant), and segmented (sample, position, value).
    
    
    Dense matrices can be genotypic or phenotypic. Phenotypic matrices have
    associated field metadata (descriptive names, codes, etc.).
    
    Genotypic matricies may have an associated probeMap, which maps probes to
    genomic locations. If a matrix has hugo probeMap, the probes themselves
    are gene names. Otherwise, a probeMap is used to map a gene location to a
    set of probes.

PACKAGE CONTENTS
    convert
    example
    jupyter
    xenaAPI
    xenaQuery


In [9]:
host = 'https://toil.xenahubs.net'

### Our dataset is the RSEM expected_count (DESeq2 standardized) 
### i.e. 'TCGA-GTEx-TARGET-gene-exp-counts.deseq2-normalized.log2' from the above

In [10]:
dataset = 'TCGA-GTEx-TARGET-gene-exp-counts.deseq2-normalized.log2'

In [11]:
xena.dataset_samples_n_dense_matrix(host, dataset) #That is the full number of samples

[19039]

In [12]:
samples = xena.dataset_samples(host, dataset,19039) # All of the samples

In [13]:
len(samples)

19039

In [14]:
samples[10]

'TCGA-DX-A7EO-01'

### We will split our data into TCGA , TARGET and GTEX samples. This will be done in order for us to be able to load datasets as well as make the analysis better

In [15]:
tcga = []
target = []
gtex = []

for i in range(len(samples)):
    if samples[i].startswith('TCGA'):
        tcga.append(samples[i])
    elif samples[i].startswith('TARGET'):
        target.append(samples[i])
    else:
        gtex.append(samples[i])
            



In [16]:
len(tcga)

10530

In [17]:
len(target)

734

In [18]:
len(gtex)

7775

In [19]:
genes = xena.dataset_field(host, dataset) #all of the genes

In [20]:
len(genes)

60499

In [21]:
#Let's create a function that we pass batches of genes and samples and creates a dataframe 

def pos(samples, genes):
    cohort = 'TCGA TARGET GTEx'
    host = 'https://toil.xenahubs.net'
    dataset = 'TCGA-GTEx-TARGET-gene-exp-counts.deseq2-normalized.log2'
    
    positions = xena.dataset_probe_values(host, dataset, samples, genes) #this returns the normalized position of the genes
    gene_pos = dict(zip(genes, positions[1]))
    df = pd.DataFrame.from_dict(gene_pos)
    df.index = samples
    
    return df

In [22]:
chunks = [genes[x:x+1000] for x in range(0, len(genes), 1000)] #let's take chuncks of 1000 genes at a time

In [23]:
len(chunks)

61

In [24]:
df1 = pos(samples, genes[9800:9900])

In [25]:
df1.head()

,ENSG00000154719.13,ENSG00000154721.14,ENSG00000154723.12,ENSG00000154727.10,ENSG00000154734.14,ENSG00000154736.5,ENSG00000154743.17,ENSG00000154760.13,ENSG00000154764.5,ENSG00000154767.14,...,ENSG00000155629.14,ENSG00000155636.14,ENSG00000155640.6,ENSG00000155657.23,ENSG00000155659.14,ENSG00000155660.10,ENSG00000155666.11,ENSG00000155714.13,ENSG00000155719.16,ENSG00000155729.12
TCGA-AD-5900-01,10.100,8.004,11.78,10.34,9.243,6.769,8.903,9.766,0.0000,10.950,...,11.080,8.503,8.026,6.058,9.794,13.72,7.234,1.551,3.413,9.271
TCGA-BP-4968-01,10.600,9.964,12.56,11.11,12.880,9.522,8.345,11.270,2.3690,11.260,...,11.440,9.026,6.116,8.028,9.687,13.67,8.549,2.116,6.310,10.040
TCGA-NG-A4VU-01,10.210,10.730,12.82,11.04,11.350,10.420,8.149,7.096,5.5670,9.727,...,9.689,9.037,7.748,16.150,8.179,12.84,7.564,2.150,4.545,8.947
TCGA-CG-4305-01,10.200,8.962,11.79,11.11,10.100,8.454,10.140,10.170,1.4770,10.730,...,11.320,7.934,9.116,8.587,11.260,13.80,7.140,1.878,4.302,9.577
TCGA-AO-A03M-01,9.494,9.442,11.82,10.82,11.680,8.287,7.884,8.230,0.6768,10.990,...,10.400,8.733,9.143,7.361,9.391,14.79,7.638,0.000,4.533,9.623


In [28]:
df1.describe()

,ENSG00000154719.13,ENSG00000154721.14,ENSG00000154723.12,ENSG00000154727.10,ENSG00000154734.14,ENSG00000154736.5,ENSG00000154743.17,ENSG00000154760.13,ENSG00000154764.5,ENSG00000154767.14,...,ENSG00000155629.14,ENSG00000155636.14,ENSG00000155640.6,ENSG00000155657.23,ENSG00000155659.14,ENSG00000155660.10,ENSG00000155666.11,ENSG00000155714.13,ENSG00000155719.16,ENSG00000155729.12
count,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,...,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000,19039.000000
mean,9.216853,8.893895,11.423807,10.287863,11.049464,7.910874,8.506836,8.472479,3.440166,10.856846,...,8.838714,8.388410,7.110810,8.789456,8.731416,12.326441,7.433425,1.930716,3.616432,9.241333
std,0.964574,1.957889,1.098617,0.927541,2.353984,2.382528,0.823556,2.166364,3.372494,0.843610,...,2.491209,0.737831,1.353833,2.617947,2.492504,1.839742,1.043842,1.407380,1.957262,0.865747
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.505000,7.790000,10.720000,9.663500,9.874500,6.663000,8.085000,6.862500,0.000000,10.390000,...,7.031000,7.954000,6.107000,7.493000,7.307500,10.830000,6.731000,1.007500,2.302000,8.673000
50%,9.286000,9.085000,11.480000,10.220000,11.270000,8.174000,8.472000,8.499000,2.330000,10.850000,...,8.911000,8.374000,6.956000,8.524000,8.887000,12.510000,7.401000,1.824000,3.660000,9.203000
75%,9.890000,10.210000,12.120000,10.930000,12.605000,9.446000,8.954000,10.170000,6.031000,11.320000,...,10.600000,8.873000,8.010000,9.362000,10.410000,13.820000,8.047000,2.681000,4.909000,9.866000
max,13.100000,15.510000,16.740000,13.990000,17.040000,16.480000,12.850000,14.840000,15.420000,14.350000,...,16.520000,10.800000,12.450000,24.160000,16.810000,17.530000,13.910000,9.934000,12.350000,12.490000


### We will work with batches of 1000 features. According to:
### Jaskowiak, P. A., Costa, I. G., & Campello, R. J. G. B. (2018). Clustering of RNA-Seq samples: Comparison study on cancer data. Methods, 132, 42–49. https://doi.org/10.1016/j.ymeth.2017.07.023 ,
### using samples with 1000 features at a time, can lead to superior results in clustering. We will begin our analysis with datasets of 1000 features at a time

### We will focus our analysis in tcga samples for the time being

### According to Vidman, L., Källberg, D., &#38; Rydén, P. (2019). Cluster analysis on high dimensional RNA-seq data with applications to cancer research - An evaluation study. PLoS ONE, 1412). https://doi.org/10.1371/journal.pone.0219102 genes  that either showed
### 1)  small variation between samples (i.e. standard deviation less than  0.5) 
### 2) or were 0 everywhere

In [29]:
#Function for use if we decide to get all of the data
nums = list(range(0,61,1))

names = ['df'+str(nums[x]) for x in range(len(nums))]


for i in range(len(names)):
    names[i] = pos(tcga, chunks[i])
    names[i] = names[i].loc[:, (names[i]!= 0).any(axis=0)]
    names[i].drop(names[i].std()[names[i].std() < 0.5].index.values, axis=1)

In [30]:
dataframes = []
for i in range(len(names)):
    dataframes.append(names[i])

In [31]:
df = pd.concat([df.set_index(df.index) for df in dataframes], ignore_index=False, axis=1)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10530 entries, TCGA-AD-5900-01 to TCGA-FU-A3HZ-01
Columns: 57060 entries, ENSG00000000003.14 to sampleID
dtypes: float64(57059), int64(1)
memory usage: 4.5+ GB


In [33]:
df.head()

,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,...,ENSG00000282785.1,ENSG00000282787.1,ENSG00000282793.1,ENSG00000282795.1,ENSG00000282798.1,ENSG00000282804.1,ENSG00000282807.1,ENSG00000282815.1,ENSG00000282816.1,sampleID
TCGA-AD-5900-01,10.670,0.000,10.38,9.036,8.496,8.563,10.050,11.44,10.860,9.896,...,1.551,1.962,0.9745,0.000,3.538,0.0000,0.0000,0.0000,0.0,0
TCGA-BP-4968-01,11.580,5.298,10.51,9.421,7.842,10.280,12.410,12.69,10.940,10.290,...,0.000,2.116,0.8746,0.000,2.773,0.8746,0.8746,0.8746,0.0,1
TCGA-NG-A4VU-01,9.693,3.262,11.00,9.155,9.683,7.383,8.848,11.19,9.806,11.180,...,0.000,2.977,3.2620,0.000,2.977,0.0000,0.0000,0.0000,0.0,2
TCGA-CG-4305-01,10.260,0.000,11.10,9.465,9.078,9.173,12.320,12.09,12.220,11.040,...,6.791,1.878,0.0000,0.000,6.066,0.0000,0.0000,0.0000,0.0,3
TCGA-AO-A03M-01,10.100,1.997,11.74,10.380,9.157,8.509,11.630,11.80,10.580,11.680,...,0.000,3.230,0.0000,1.136,4.050,0.6768,0.0000,3.8250,0.0,4


In [34]:
len(df)

10530

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10530 entries, TCGA-AD-5900-01 to TCGA-FU-A3HZ-01
Columns: 57060 entries, ENSG00000000003.14 to sampleID
dtypes: float64(57059), int64(1)
memory usage: 4.5+ GB


## Now we will only keep the genes that are protein coding. Using the ensemble IDs from HGNC we were able to retrieve a dataset that contains only protein coding genes

In [33]:
protein_cd = pd.read_csv('../Data/protein_coding_genes.csv')

In [34]:
protein_cd.head()

,Ensembl ID(supplied by Ensembl)
0,ENSG00000121410
1,ENSG00000148584
2,ENSG00000175899
3,ENSG00000166535
4,ENSG00000184389


In [35]:
protein_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19148 entries, 0 to 19147
Data columns (total 1 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Ensembl ID(supplied by Ensembl)  19148 non-null  object
dtypes: object(1)
memory usage: 149.7+ KB


In [36]:
genes_to_keep = list(protein_cd['Ensembl ID(supplied by Ensembl)']) #make a list containing protein coding genes

In [38]:
all_cols = list(df.columns)

NameError: name 'df' is not defined

In [61]:
all_cols[0].split('.')[0]

'ENSG00000000003'

In [63]:
our_genes = []
for i in range(len(all_cols)):
    if all_cols[i].split('.')[0] in genes_to_keep:
        our_genes.append(all_cols[i])
   


In [65]:
len(our_genes)

19037

In [66]:
df_final = df[our_genes]

In [67]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10530 entries, TCGA-AD-5900-01 to TCGA-FU-A3HZ-01
Columns: 19037 entries, ENSG00000000003.14 to ENSG00000282815.1
dtypes: float64(19037)
memory usage: 1.5+ GB


In [69]:
df_final.to_csv('../Data/data.csv')

In [ ]:
## We will now repeat the above procedure for the GTEX samples

In [29]:
#Function for use if we decide to get all of the data
num = list(range(0,61,1))

names_gtex = ['df'+str(num[x]) for x in range(len(num))]


for i in range(len(names_gtex)):
    names_gtex[i] = pos(gtex, chunks[i])
    names_gtex[i] = names_gtex[i].loc[:, (names_gtex[i]!= 0).any(axis=0)]
    names_gtex[i].drop(names_gtex[i].std()[names_gtex[i].std() < 0.5].index.values, axis=1)

In [30]:
dataframes_gtex = []
for i in range(len(names_gtex)):
    dataframes_gtex.append(names_gtex[i])

In [31]:
df_gtex = pd.concat([df.set_index(df.index) for df in dataframes_gtex], ignore_index=False, axis=1)

In [32]:
df_gtex.head()

,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,...,ENSG00000282785.1,ENSG00000282787.1,ENSG00000282793.1,ENSG00000282795.1,ENSG00000282798.1,ENSG00000282804.1,ENSG00000282807.1,ENSG00000282815.1,ENSG00000282816.1,sampleID
GTEX-UTHO-1226-SM-3GAEE,9.124,0.000,9.375,8.514,7.637,10.110,15.720,10.170,8.951,10.190,...,0.0,0.000,0.0,0.0,3.265,0.0000,0.0,0.000,0.0,10530
GTEX-146FH-1726-SM-5QGQ2,11.360,0.000,9.341,9.245,7.970,7.402,10.620,8.902,10.260,9.446,...,0.0,0.000,0.0,0.0,4.704,0.0000,0.0,0.000,0.0,10531
GTEX-QDT8-0126-SM-48TZ1,8.926,6.679,9.009,9.016,7.294,9.091,12.820,9.356,9.262,9.365,...,0.0,2.122,0.0,0.0,3.573,0.0000,0.0,2.122,0.0,10532
GTEX-QCQG-1326-SM-48U24,9.862,1.436,9.138,8.547,7.232,8.685,7.618,10.200,9.925,9.481,...,0.0,3.491,0.0,0.0,3.694,0.8901,0.0,0.000,0.0,10533
GTEX-WZTO-2926-SM-3NM9I,8.429,2.591,9.153,8.497,6.185,7.117,9.470,8.746,9.765,9.334,...,0.0,0.000,0.0,0.0,3.799,0.0000,0.0,1.284,0.0,10534


In [39]:
all_cols_gtex = list(df_gtex.columns)

In [41]:
our_genes_gtex = []
for i in range(len(all_cols_gtex)):
    if all_cols_gtex[i].split('.')[0] in genes_to_keep:
        our_genes_gtex.append(all_cols_gtex[i])
   


In [42]:
#again keep only the protein coding genes

df_gtex = df_gtex[our_genes_gtex]

In [43]:
df_gtex.head()

,ENSG00000000003.14,ENSG00000000005.5,ENSG00000000419.12,ENSG00000000457.13,ENSG00000000460.16,ENSG00000000938.12,ENSG00000000971.15,ENSG00000001036.13,ENSG00000001084.10,ENSG00000001167.14,...,ENSG00000280314.1,ENSG00000280670.2,ENSG00000280789.1,ENSG00000280969.1,ENSG00000281106.2,ENSG00000281991.1,ENSG00000282419.1,ENSG00000282608.1,ENSG00000282757.1,ENSG00000282815.1
GTEX-UTHO-1226-SM-3GAEE,9.124,0.000,9.375,8.514,7.637,10.110,15.720,10.170,8.951,10.190,...,0.0,5.260,9.723,0.0000,6.241,4.746,0.0,7.460,0.0,0.000
GTEX-146FH-1726-SM-5QGQ2,11.360,0.000,9.341,9.245,7.970,7.402,10.620,8.902,10.260,9.446,...,0.0,6.454,9.839,0.0000,2.861,8.651,0.0,5.521,0.0,0.000
GTEX-QDT8-0126-SM-48TZ1,8.926,6.679,9.009,9.016,7.294,9.091,12.820,9.356,9.262,9.365,...,0.0,4.563,9.641,0.0000,2.780,8.876,0.0,6.825,0.0,2.122
GTEX-QCQG-1326-SM-48U24,9.862,1.436,9.138,8.547,7.232,8.685,7.618,10.200,9.925,9.481,...,0.0,7.507,10.710,0.0000,2.614,7.030,0.0,5.300,0.0,0.000
GTEX-WZTO-2926-SM-3NM9I,8.429,2.591,9.153,8.497,6.185,7.117,9.470,8.746,9.765,9.334,...,0.0,7.627,10.450,0.7804,5.850,6.773,0.0,4.961,0.0,1.284


In [44]:
df_gtex.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7775 entries, GTEX-UTHO-1226-SM-3GAEE to GTEX-12BJ1-0426-SM-5FQSO
Columns: 19036 entries, ENSG00000000003.14 to ENSG00000282815.1
dtypes: float64(19036)
memory usage: 1.1+ GB


In [46]:
df_gtex.to_csv('../Data/df_gtex.csv')